In [337]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
train_d=pd.read_csv("train.csv")
test_d=pd.read_csv("test.csv")

to know what is the best feauters to dealing with

In [338]:
corr = train_d[[i for i in train_d.select_dtypes(
    include=("int64", "float64")).columns]].corr()
                

print(corr['SalePrice'].sort_values(ascending=False)[:10], '\n')


SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
Name: SalePrice, dtype: float64 



how much missing values in columns

In [339]:
# for training dataset
t_nul_col_with_number_of_misssing=[train_d[i].isnull().sum() for i in train_d.columns if train_d[i].isnull().any()]
t_nul_col_with_name = [i for i in train_d.columns if train_d[i].isnull().any()]
print(
    f"thier are {len(t_nul_col_with_number_of_misssing)} columns with missing values")
for i in range(len(t_nul_col_with_name)):

    print(f"{t_nul_col_with_number_of_misssing[i],t_nul_col_with_name[i]}")
 

thier are 19 columns with missing values
(259, 'LotFrontage')
(1369, 'Alley')
(872, 'MasVnrType')
(8, 'MasVnrArea')
(37, 'BsmtQual')
(37, 'BsmtCond')
(38, 'BsmtExposure')
(37, 'BsmtFinType1')
(38, 'BsmtFinType2')
(1, 'Electrical')
(690, 'FireplaceQu')
(81, 'GarageType')
(81, 'GarageYrBlt')
(81, 'GarageFinish')
(81, 'GarageQual')
(81, 'GarageCond')
(1453, 'PoolQC')
(1179, 'Fence')
(1406, 'MiscFeature')


In [340]:
col_to_del=["Alley","PoolQC","Fence","MiscFeature",'LotFrontage',"BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","FireplaceQu","GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond"]
train_d=train_d.drop(col_to_del,axis=1)
# train_d=train_d.drop(["SalePrice"],axis=1)
test_d=test_d.drop(col_to_del,axis=1)


here we are imputing the train data because we have alot of missing values in the columns

In [341]:
my_imputer = SimpleImputer(strategy="most_frequent")
imputed_train_d=pd.DataFrame(my_imputer.fit_transform(train_d))
imputed_train_d.columns=train_d.columns

imputed_test_d = pd.DataFrame(my_imputer.fit_transform(test_d))
imputed_test_d.columns = test_d.columns
# to saving the data type of the columns
imputed_train_d = imputed_train_d.astype(train_d.dtypes.to_dict())
imputed_test_d = imputed_test_d.astype(test_d.dtypes.to_dict())


splitting the values to numeric and categorical values 

In [342]:
num_col_train=imputed_train_d.select_dtypes(include=("int64","float64")).columns
cat_col_train=imputed_train_d.select_dtypes(include=("object")).columns
#  to find the corrlation between numerica values and the predict (SalePrice) column
corr_num_col = imputed_train_d[num_col_train].corr()
print(corr_num_col['SalePrice'].sort_values(ascending=False)[:15], '\n')



SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
MasVnrArea      0.472614
Fireplaces      0.466929
BsmtFinSF1      0.386420
WoodDeckSF      0.324413
Name: SalePrice, dtype: float64 



here i will encoding the cat col data and concat it (after encoding id) with numirc col

In [343]:
OHE = OneHotEncoder(handle_unknown='ignore', sparse=False)
OHE.fit(imputed_train_d[cat_col_train])
cat_temp=pd.DataFrame(OHE.transform(imputed_train_d[cat_col_train]))
cat_temp.columns=OHE.get_feature_names_out()

# cat_temp=pd.get_dummies(imputed_train_d[cat_col_train])
num_temp=imputed_train_d[num_col_train]

# temp_d = pd.concat([cat_temp,imputed_train_d["SalePrice"]], axis=1) 
# corr = temp_d.corr()
# print(corr['SalePrice'].sort_values(ascending=False)[:20], '\n')

final_train_d = pd.concat([num_temp,cat_temp],axis=1)

c:\Users\hussam\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [344]:
num_col_test=imputed_test_d.select_dtypes(include=("int64","float64")).columns
cat_col_test=imputed_test_d.select_dtypes(include=("object")).columns


# cat_temp=pd.get_dummies(imputed_test_d[cat_col_test])

cat_temp=pd.DataFrame(OHE.transform(imputed_test_d[cat_col_test]))
num_temp=imputed_test_d[num_col_test]
cat_temp.columns=OHE.get_feature_names_out()

final_test_d = pd.concat([num_temp,cat_temp],axis=1)
print(len(final_test_d.columns))

225


In [345]:
temp_d = pd.concat([cat_temp, imputed_train_d["SalePrice"]], axis=1)
corr = temp_d.corr()
most_corr_col_cat = corr['SalePrice'].sort_values(ascending=False)


### here i will find the score of the data set

In [367]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    log_y = np.log(y)
    score = cross_val_score(
        model, X, log_y, cv=5, scoring="neg_mean_squared_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


In [370]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(final_train_d.drop(
    ["SalePrice"], axis=1), final_train_d['SalePrice'], train_size=0.8, test_size=0.2, random_state=0)

model1 = XGBRegressor(n_estimators=5000,learning_rate=0.1)
model1.fit(X_train[corr_num_col.index[:-1].append(most_corr_col_cat.index[1:])],
            y_train,
            early_stopping_rounds=5,
           eval_set=[(X_valid[corr_num_col.index[:-1].append(most_corr_col_cat.index[1:])],
                       y_valid)],
           verbose=False)


score_dataset(X_train,y_train)

c:\Users\hussam\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.13957235728211956

In [358]:
mse = mean_absolute_error(y_valid, model1.predict(
    X_valid[corr_num_col.index[:-1].append(most_corr_col_cat.index[1:])]))
print(mse)
#


16644.47522474315


In [366]:
pd.DataFrame(model1.predict(final_test_d[corr_num_col.index[:-1].append(
    most_corr_col_cat.index[1:])]), columns=["SalePrice"] ,index=test_d['Id']).to_csv("first_prediction.csv")
